In [4]:
import pandas as pd

df = pd.read_json('/home/leesh/codeit_project/codeit-project-docker/notebooks/origin_data/2023-05-30.json')

In [5]:
df.head()

,Asia/Seoul,id,event_key,session_id,device_id,user_id,value,user_properties,hackle_properties,event_properties
0,2023-05-31T00:01:06.774Z,ea61928e-0642-4765-a5a7-6bf8f573784b,$session_start,VAb6ai91iKPZZQhGKenhHZpAG5B2,6DA6A8DC-1E61-4643-963A-ED3D782BE77A,NaN,0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1685458866774.78343b3e'}
1,2023-05-31T00:02:20.534Z,773f4cd7-00d1-4b79-bfe6-59788a145d01,$session_start,fe556647-de19-43ff-9152-b3c38a8b4d26,fe556647-de19-43ff-9152-b3c38a8b4d26,NaN,0,{},"{'language': 'ko', 'isapp': 'true', 'osversion...",{'sessionid': '1685458940534.47bbaa49'}
2,2023-05-31T00:02:46.447Z,8998b069-72fe-4a69-9b9c-310e7f631625,$session_start,LsAqqCTVtQPpevfSHXI8oRepVs73,5D47D549-EF22-41F3-A0D2-AD9EC1D21A2B,NaN,0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1685458966447.17499524'}
3,2023-05-31T00:03:50.029Z,bcc3c519-7de5-4fd4-a66c-ab348d73de61,$session_start,NBfi1jm5FKfB6We7YfVRHFgeJY83,FD419DE8-B3CE-4630-B234-959ADA5F47CF,NaN,0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1685459030029.9ef625ea'}
4,2023-05-31T00:04:12.317Z,14b3131e-3c30-4457-a52d-215441d8df1d,$session_start,3CBFC179-1BB1-40A2-84EF-C39EFBD40C14,3CBFC179-1BB1-40A2-84EF-C39EFBD40C14,NaN,0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1685459052317.fdf5aa30'}


In [6]:
# 주요 정보 추출
df_simple = df.copy()

# 딕셔너리 → Series로 분해 (중요한 컬럼만)
df_simple['language'] = df_simple['hackle_properties'].apply(lambda x: x.get('language'))
df_simple['devicemodel'] = df_simple['hackle_properties'].apply(lambda x: x.get('devicemodel'))
df_simple['sessionid'] = df_simple['event_properties'].apply(lambda x: x.get('sessionid'))


# 1. UTC 기준으로 파싱
df_simple['event_datetime'] = pd.to_datetime(df_simple['Asia/Seoul'], utc=True)

# 2. 한국시간(Asia/Seoul)으로 타임존 변환
df_simple['event_datetime'] = df_simple['event_datetime'].dt.tz_convert('Asia/Seoul')

# 최종 확인용 컬럼만 선택
df_simple = df_simple[['event_datetime','session_id','device_id', 'event_key', 'devicemodel', 'language']]

In [7]:
df_simple['event_datetime'] = df_simple['event_datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_simple.head()

,event_datetime,session_id,device_id,event_key,devicemodel,language
0,2023-05-31 09:01:06,VAb6ai91iKPZZQhGKenhHZpAG5B2,6DA6A8DC-1E61-4643-963A-ED3D782BE77A,$session_start,"iPhone15,2",ko-KR
1,2023-05-31 09:02:20,fe556647-de19-43ff-9152-b3c38a8b4d26,fe556647-de19-43ff-9152-b3c38a8b4d26,$session_start,SM-G991N,ko
2,2023-05-31 09:02:46,LsAqqCTVtQPpevfSHXI8oRepVs73,5D47D549-EF22-41F3-A0D2-AD9EC1D21A2B,$session_start,"iPhone15,2",ko-KR
3,2023-05-31 09:03:50,NBfi1jm5FKfB6We7YfVRHFgeJY83,FD419DE8-B3CE-4630-B234-959ADA5F47CF,$session_start,"iPhone9,3",ko-KR
4,2023-05-31 09:04:12,3CBFC179-1BB1-40A2-84EF-C39EFBD40C14,3CBFC179-1BB1-40A2-84EF-C39EFBD40C14,$session_start,"iPhone14,7",ko-KR


In [8]:
df_simple.to_csv('/home/leesh/codeit_project/codeit-project-docker/notebooks/data/df_230530.csv',index=False)